## Strain minimization of LaueTools
### Useful for users to understand the strain minimization

### pip install lauetoolsnn 
### use the above command to install lauetoolsnn to be able to use this notebook

In [1]:
## import required functions
import numpy as np
import os
from lauetoolsnn.utils_lauenn import resource_path

base_lib_path = resource_path('')
import json
## Load the json of material and extinctions
with open(os.path.join(base_lib_path, 'lauetools/material.json'),'r') as f:
    dict_Materials = json.load(f)
## Modify the dictionary values to add new entries
dict_Materials["Cu_test"] = ["Cu_test", [5, 5, 5, 90, 90, 90], "fcc"]
dict_Materials["Cu_test_strain"] = ["Cu_test_strain", [5., 5.02, 4.98, 90.0+np.random.rand(), 90.0+np.random.rand(), 90.0+np.random.rand()], "fcc"]
## dump the json back with new values
with open(os.path.join(base_lib_path, 'lauetools/material.json'), 'w') as fp:
    json.dump(dict_Materials, fp)
    
### AFTER THIS STEP, RESTART THE KERNAL TO TAKE THIS NEW MAT INTO EFFECT

adjustText library not installed


In [14]:
## load Lauetoolsnn Library
import lauetoolsnn.lauetools.dict_LaueTools as dictLT
import lauetoolsnn.lauetools.generaltools as GT
import lauetoolsnn.lauetools.CrystalParameters as CP
import lauetoolsnn.lauetools.lauecore as LT
import lauetoolsnn.lauetools.LaueGeometry as Lgeo
import lauetoolsnn.lauetools.readmccd as RMCCD
import lauetoolsnn.lauetools.FitOrient as FitO
import lauetoolsnn.lauetools.LaueGeometry as F2TC

mat_exp = "Cu_test_strain" ## No longer cubic
mat_sim = "Cu_test"  ## reference is Cubic
## Lets create a fake strained Experimental laue pattern with Cu_test_strain material
detectorparameters = [79.5, 950, 950, 0.342, 0.482]
pixelsize = 0.0734

######################### FAKE EXPERIMENTAL PATTERN  ######################################
UBmatrix_exp = np.eye(3)
## lets rotate the fake pattern matrix by few degrees
rot_angX = np.deg2rad(0.1)
rot_angY = np.deg2rad(0.0)
rot_angZ = np.deg2rad(0.0)
mat1 = np.array([[np.cos(rot_angY), 0, np.sin(rot_angY)], [0, 1, 0], [-np.sin(rot_angY), 0, np.cos(rot_angY)]])
mat2 = np.array([[1, 0, 0], [0, np.cos(rot_angX), np.sin(rot_angX)], [0, np.sin(-rot_angX), np.cos(rot_angX)]])
mat3 = np.array([[np.cos(rot_angZ), -np.sin(rot_angZ), 0], [np.sin(rot_angZ), np.cos(rot_angZ), 0], [0, 0, 1]])
deltamat = np.dot(mat3, np.dot(mat2, mat1))
UBmatrix_exp = np.dot(deltamat, UBmatrix_exp)
grain = CP.Prepare_Grain(mat_exp, UBmatrix_exp)
s_tth, s_chi, Miller_ind_exp, exp_posx, exp_posy, _ = LT.SimulateLaue_full_np(grain, 5, 22,
                                                    detectorparameters,
                                                    pixelsize=pixelsize,
                                                    dim=(2018,2016),
                                                    detectordiameter=pixelsize*2018*1.3,
                                                    removeharmonics=1)

######################### SIMULATED PATTERN  ######################################
UBmatrix = np.eye(3)
grain = CP.Prepare_Grain(mat_sim, UBmatrix)
Twicetheta, Chi, Miller_ind, s_posx, s_posy, s_E = LT.SimulateLaue_full_np(grain, 5, 22,
                                                                            detectorparameters,
                                                                            pixelsize=pixelsize,
                                                                            dim=(2018,2016),
                                                                            detectordiameter=pixelsize*2018*1.3,
                                                                            removeharmonics=1)

## test if new material is added or not
UBmatrix = np.round(UBmatrix,4)
print(dictLT.dict_Materials["Cu_test_strain"])
print("UBmatrix of fake strained exp laue pattern is:", UBmatrix_exp[:,0], UBmatrix_exp[:,1], UBmatrix_exp[:,2])
print(dictLT.dict_Materials["Cu_test"])
print("UBmatrix of reference laue pattern is:", UBmatrix[:,0], UBmatrix[:,1], UBmatrix[:,2])
print()
print("We will start with the non strained Cu as initial (reference) material, it should be changed to match the values of Fake Exp Laue patterns, which has some strain")

['Cu_test_strain', [5.0, 5.02, 4.98, 90.48570992919974, 90.35046242365307, 90.74507566406014], 'fcc']
UBmatrix of fake strained exp laue pattern is: [1. 0. 0.] [ 0.                 0.999998476913288 -0.001745328365898] [0.                0.001745328365898 0.999998476913288]
['Cu_test', [5, 5, 5, 90, 90, 90], 'fcc']
UBmatrix of reference laue pattern is: [1. 0. 0.] [0. 1. 0.] [0. 0. 1.]

We will start with the non strained Cu as initial (reference) material, it should be changed to match the values of Fake Exp Laue patterns, which has some strain


In [15]:
### print how far both laue patterns are from each other in angle and pixel space
print("Exp", "Sim")
print(s_tth[0], Twicetheta[0])
print(s_chi[0], Chi[0])
print(exp_posx[0], s_posx[0])
print(exp_posy[0], s_posy[0])
print(Miller_ind_exp[0], Miller_ind[0])

Exp Sim
109.88505944385517 109.47122063449068
-44.42601204966866 -45.0
2019.5034999558425 2040.9533820086704
402.2656549906868 409.499614321252
[-4 -2  2] [-4 -2  2]


### Functions used for strain computation by varying lattice parameter

In [16]:
## Different functions
def getProximity(TwicethetaChi, data_theta, data_chi, data_hkl, angtol=0.5):
    """This functions gives the indices of all the experimetnal spots that are close to the simulated spots"""
    # theo simul data
    theodata = np.array([TwicethetaChi[0] / 2.0, TwicethetaChi[1]]).T
    # exp data
    sorted_data = np.array([data_theta, data_chi]).T
    table_dist = GT.calculdist_from_thetachi(sorted_data, theodata)
    prox_table = np.argmin(table_dist, axis=1)
    allresidues = np.amin(table_dist, axis=1)
    very_close_ind = np.where(allresidues < angtol)[0]
    List_Exp_spot_close = []
    Miller_Exp_spot = []
    if len(very_close_ind) > 0:
        for theospot_ind in very_close_ind:  # loop over theo spots index
            List_Exp_spot_close.append(prox_table[theospot_ind])
            Miller_Exp_spot.append(data_hkl[theospot_ind])
    else:
        return [],[],[]
    # removing exp spot which appears many times(close to several simulated spots of one grain)--------------
    arrayLESC = np.array(List_Exp_spot_close, dtype=float)
    sorted_LESC = np.sort(arrayLESC)
    diff_index = sorted_LESC - np.array(list(sorted_LESC[1:]) + [sorted_LESC[0]])
    toremoveindex = np.where(diff_index == 0)[0]
    if len(toremoveindex) > 0:
        # index of exp spot in arrayLESC that are duplicated
        ambiguous_exp_ind = GT.find_closest(np.array(sorted_LESC[toremoveindex], dtype=float), arrayLESC, 0.1)[1]
        for ind in ambiguous_exp_ind:
            Miller_Exp_spot[ind] = None
    ProxTablecopy = np.copy(prox_table)
    for theo_ind, exp_ind in enumerate(prox_table):
        where_th_ind = np.where(ProxTablecopy == exp_ind)[0]
        if len(where_th_ind) > 1:
            for indy in where_th_ind:
                ProxTablecopy[indy] = -prox_table[indy]
            closest = np.argmin(allresidues[where_th_ind])
            ProxTablecopy[where_th_ind[closest]] = -ProxTablecopy[where_th_ind[closest]]
    singleindices = []
    refine_indexed_spots = {}
    # loop over close exp. spots
    for k in range(len(List_Exp_spot_close)):
        exp_index = List_Exp_spot_close[k]
        if not singleindices.count(exp_index):
            singleindices.append(exp_index)
            theo_index = np.where(ProxTablecopy == exp_index)[0]
            if (len(theo_index) == 1):  # only one theo spot close to the current exp. spot
                refine_indexed_spots[exp_index] = [exp_index, theo_index, Miller_Exp_spot[k]]
            else:  # recent PATCH:
                closest_theo_ind = np.argmin(allresidues[theo_index])
                if allresidues[theo_index][closest_theo_ind] < angtol:
                    refine_indexed_spots[exp_index] = [exp_index, theo_index[closest_theo_ind], Miller_Exp_spot[k]]
    listofpairs = []
    linkExpMiller = []
    selectedAbsoluteSpotIndices = np.arange(len(data_theta))
    for val in list(refine_indexed_spots.values()):
        if val[2] is not None:
            localspotindex = val[0]
            if not isinstance(val[1], (list, np.ndarray)):
                closetheoindex = val[1]
            else:
                closetheoindex = val[1][0]
            absolute_spot_index = selectedAbsoluteSpotIndices[localspotindex]
            listofpairs.append([absolute_spot_index, closetheoindex])  # Exp, Theo,  where -1 for specifying that it came from automatic linking
            linkExpMiller.append([float(absolute_spot_index)] + [float(elem) for elem in val[2]])  # float(val) for further handling as floats array
    linkedspots_link = np.array(listofpairs)
    linkExpMiller_link = linkExpMiller
    return linkedspots_link, linkExpMiller_link

def error_function_on_demand_strain(param_strain, DATA_Q, nspots,
                                    pixX, pixY, initrot=np.eye(3), Bmat=np.eye(3),
                                    verbose=0, pixelsize=165.0 / 2048., weights=None,
                                    kf_direction="Z>0"):
    DEG = np.pi / 180.0
    RAD = 1.0 / DEG
    ## Building rotation matrix along X, Y and Z
    a1 = param_strain[5] * DEG
    mat1 = np.array([[np.cos(a1), 0, np.sin(a1)], [0, 1, 0], [-np.sin(a1), 0, np.cos(a1)]])
    a2 = param_strain[6] * DEG
    mat2 = np.array([[1, 0, 0], [0, np.cos(a2), np.sin(a2)], [0, np.sin(-a2), np.cos(a2)]])
    a3 = param_strain[7] * DEG
    mat3 = np.array([[np.cos(a3), -np.sin(a3), 0], [np.sin(a3), np.cos(a3), 0], [0, 0, 1]])
    deltamat = np.dot(mat3, np.dot(mat2, mat1))

    # building B mat with proposed lattice parameters (except 'a' is fixed to 1)
    varyingstrain = np.array([[1.0, param_strain[2], param_strain[3]], [0, param_strain[0], param_strain[4]],  [0, 0, param_strain[1]]])
    newmatrix = np.dot(np.dot(deltamat, initrot), varyingstrain)
    
    X, Y, _, _ = xy_from_Quat(DATA_Q, nspots,
                                initrot=newmatrix, vecteurref=Bmat,
                                pixelsize=pixelsize, kf_direction=kf_direction)
    distanceterm = np.sqrt((X - pixX) ** 2 + (Y - pixY) ** 2)
    if weights is not None:
        allweights = np.sum(weights)
        distanceterm = distanceterm * weights / allweights
    if verbose:
        return distanceterm, deltamat, newmatrix
    else:
        print("SSE:", np.sum(distanceterm))
        return distanceterm
    
def xy_from_Quat(DATA_Q, nspots, initrot=None,  vecteurref=np.eye(3),
                 pixelsize=165.0 / 2048, kf_direction="Z>0"):
    """
    compute x and y pixel positions of Laue spots given hkl list
    """
    # selecting nspots of DATA_Q
    DATAQ = np.take(DATA_Q, nspots, axis=0)
    trQ = np.transpose(DATAQ)  # np.array(Hs, Ks, Ls) for further computations
    # R is a pure rotation
    # dot(R,Q)=initrot # Q may be viewed as lattice distortion
    R = initrot # keep UB matrix rotation + distorsion
    # initial lattice rotation and distorsion (/ cubic structure)  q = U*B * Q
    trQ = np.dot(np.dot(R, vecteurref), trQ)
    # results are qx,qy,qz
    matfromQuat = np.eye(3)
    Qrot = np.dot(matfromQuat, trQ)  # lattice rotation due to quaternion
    Qrotn = np.sqrt(np.sum(Qrot ** 2, axis=0))  # norms of Q vectors
    twthe, chi = F2TC.from_qunit_to_twchi(1.*Qrot / Qrotn)
    X, Y, theta = F2TC.calc_xycam_from2thetachi(twthe, chi, detectorparameters, verbose=0, pixelsize=pixelsize, kf_direction=kf_direction)
    return X, Y, theta, R

### Least square optimization to refine the lattice parameters (+ orientation) to match the experimental pattern

#### Least square varies the Lattice parameters and we compute the new Laue pattern for the new lattice parameters
#### Then the radial distance between the newly computed Laue spots and Experimental spots is used as minimization during least square

In [17]:
from scipy.optimize import leastsq

## best guess UB matrix
UBmat = UBmatrix #np.eye(3)

## reference pattern
lattice_params_sim = dictLT.dict_Materials[mat_sim][1]
B0matrix = CP.calc_B_RR(lattice_params_sim)

## get proximity for exp and theo spots
linkedspots_link, linkExpMiller_link = getProximity(np.array([Twicetheta, Chi]),  # warning array(2theta, chi) ## Simulated
                                                    s_tth/2.0, s_chi, # warning theta, chi for exp
                                                    Miller_ind, angtol=0.5)

if len(linkedspots_link) < 8:
    print("Not enough spots to use Least square fitting")
else:
    arraycouples = np.array(linkedspots_link)
    exp_indices = np.array(arraycouples[:, 0], dtype=np.int)
    sim_indices = np.array(arraycouples[:, 1], dtype=np.int)

    nb_pairs = len(exp_indices)
    Data_Q = np.array(linkExpMiller_link)[:, 1:]
    sim_indices = np.arange(nb_pairs)  # for fitting function this must be an arange...

    pixX = np.take(exp_posx, exp_indices)
    pixY = np.take(exp_posy, exp_indices)

    starting_orientmatrix = np.copy(UBmat)
    # ----------------------------------
    #  refinement model
    # ----------------------------------
    # -------------------------------------------------------
    # strain & orient
    initial_values = np.array([1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    
    residues, deltamat, newmatrix = error_function_on_demand_strain(
                                                                    initial_values,
                                                                    Data_Q,
                                                                    sim_indices,
                                                                    pixX,
                                                                    pixY,
                                                                    initrot=starting_orientmatrix,
                                                                    Bmat=B0matrix,
                                                                    verbose=1,
                                                                    pixelsize=pixelsize,
                                                                    weights=None,
                                                                    kf_direction='Z>0')
    init_mean_residues = np.copy(np.mean(residues))

    # setting  keywords of error_function_on_demand_strain during the fitting because leastsq handle only *args but not **kwds
    error_function_on_demand_strain.__defaults__ = (starting_orientmatrix,
                                                    B0matrix,
                                                    0,
                                                    pixelsize,
                                                    None,
                                                    'Z>0')
    # LEASTSQUARE
    res = leastsq(error_function_on_demand_strain,
                    initial_values,
                    args=(Data_Q, sim_indices, pixX, pixY),
                    maxfev=5000,
                    full_output=1,
                    xtol=1.0e-11,
                    epsfcn=0.0)
    strain_sol = res[0]
    if res[-1] not in (1, 2, 3, 4, 5):
        results = None
        print("Results are None, no strain refinement done")
    else:
        results = strain_sol
    lattice_part = results[:5]
    orient_part = results[5:]
    residues, deltamat, newmatrix = error_function_on_demand_strain(
                                                                    results,
                                                                    Data_Q,
                                                                    sim_indices,
                                                                    pixX,
                                                                    pixY,
                                                                    initrot=starting_orientmatrix,
                                                                    Bmat=B0matrix,
                                                                    verbose=1,
                                                                    pixelsize=pixelsize,
                                                                    weights=None,
                                                                    kf_direction='Z>0')
    final_residues = np.copy(np.mean(residues))
    UBmat = np.copy(newmatrix) 
    # ---------------------------------------------------------------
    # postprocessing of unit cell orientation and strain refinement
    # ---------------------------------------------------------------
    
    (devstrain, _) = CP.compute_deviatoricstrain(UBmat, B0matrix, lattice_params_sim)
    # overwrite and rescale possibly lattice lengthes
    constantlength = "a"
    lattice_parameter_direct_strain = CP.computeLatticeParameters_from_UB(UBmat, mat_sim, constantlength, dictmaterials=dictLT.dict_Materials)
    deviatoricstrain_sampleframe = CP.strain_from_crystal_to_sample_frame2(devstrain, UBmat)
    # in % already
    devstrain = np.round(devstrain * 100, decimals=3)
    deviatoricstrain_sampleframe = np.round(deviatoricstrain_sampleframe * 100, decimals=3)
    print("Lattice values from Least square (without scaling)", lattice_part)
    print("Orientation values from Least square", orient_part)
    print("Deviatoric Strain (%)", devstrain[0,:], devstrain[1,:], devstrain[2,:])
    print("Reference Lattice", dictLT.dict_Materials["Cu_test"][1])
    print("Expected Lattice distortion", np.round(dictLT.dict_Materials["Cu_test_strain"][1],4))
    print("Strain refined Lattice from reference", np.round(lattice_parameter_direct_strain,4))
    print("Initial residues", init_mean_residues)
    print("Final residues", final_residues)

SSE: 1387.9989853831157
SSE: 1387.9989853831157
SSE: 1387.9989853831157
SSE: 1387.9991513925247
SSE: 1387.9980616398552
SSE: 1387.998779281717
SSE: 1387.998554934979
SSE: 1387.9979049817048
SSE: 1387.998948598271
SSE: 1387.998966590009
SSE: 1387.9990214438637
SSE: 18.063939173922105
SSE: 18.064250604526627
SSE: 18.06259538727565
SSE: 18.063940997755303
SSE: 18.063935236776352
SSE: 18.06394315041869
SSE: 18.063938932439594
SSE: 18.063939706997054
SSE: 18.06393916161224
SSE: 0.009817796637352917
SSE: 0.009992396345852403
SSE: 0.008295673288872499
SSE: 0.009817884411561718
SSE: 0.009812674604501604
SSE: 0.009815546634911496
SSE: 0.009817796390304466
SSE: 0.009817200658666196
SSE: 0.009817796660076105
SSE: 0.0018603135106324562
SSE: 0.0016740052726688568
SSE: 0.0026629740681045457
SSE: 0.0018619964431318904
SSE: 0.001859151312830089
SSE: 0.0018607223787049218
SSE: 0.0018603135124416095
SSE: 0.0018601975170449254
SSE: 0.0018603135149770742
SSE: 0.0037499751243041625
SSE: 0.00374997512430416